In [1]:
%load_ext autoreload
%autoreload 2

import astropy
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt

import inspect


import apogee.tools.read as apread
import apogee.tools.path as apogee_path
from apogee.tools import bitmask
import collections

from apoNN.src.occam import Occam
from apoNN.src.datasets import ApogeeDataset,AspcapDataset
from apoNN.src.utils import get_mask_elem,dump,load
from apoNN.src.plotting import summarize_representation
import apoNN.src.vectors as vector


import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
from sklearn.decomposition import PCA

from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2,interaction_only=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

apogee_path.change_dr(16)

[(['TEFF', 'LOGG', 'LOG10VDOP', 'METALS', 'C', 'N', 'O Mg Si S Ca Ti'], ['C', 'N', 'O', 'Na', 'Mg', 'Al', 'Si', 'S', 'K', 'Ca', 'Ti', 'V', 'Mn', 'Fe', 'Ni'], ['[C/M]', '[N/M]', '[O/M]', '[Na/H]', '[Mg/M]', '[Al/H]', '[Si/M]', '[S/M]', '[K/H]', '[Ca/M]', '[Ti/M]', '[V/H]', '[Mn/H]', '[Fe/H]', '[Ni/H]'], [0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1])]


### Generating z_ae

z_ae is the raw latents for the dataset

In [2]:
elem = "Fe"
mask_elem  = get_mask_elem(elem)
autoencoder = torch.load("/share/splinter/ddm/taggingProject/apogeeFactory/outputs/guild/window/fel2/ae_20000.p",map_location=device)

/share/splinter/ddm/modules/apogee/apogee/spec/window.py:89: RuntimeWarning: invalid value encountered in greater
  mask= ((win > 0.)*(True^numpy.isnan(win))).astype('int')


In [3]:
allStar =  load("allStar_training_clean")
dataset=  AspcapDataset(filename="aspcap_training_clean",tensor_type=torch.FloatTensor,recenter=True)
n_data=10000

In [4]:
z_ae = vector.LatentVector(dataset,autoencoder,n_data)


/share/splinter/ddm/taggingProject/apogeeFactory/apoNN/src/vectors.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _,z = self.autoencoder(torch.tensor(self.dataset[idx][0]).to(device).unsqueeze(0))


### Filtering z_ae

We want to do linear regression on the set of spectra. We must thus remove all spectra for which the enhancement estimation did not converge (these spectra then have "X_FE"=-9999)

In [5]:
feh_outliercut = allStar["Fe_H"]>-5
o_outliercut = allStar["O_FE"]>-5
c_outliercut = allStar["C_FE"]>-5
na_outliercut = allStar["Na_FE"]>-5
mg_outliercut = allStar["Mg_FE"]>-5
si_outliercut = allStar["Si_FE"]>-5
al_outliercut = allStar["Al_FE"]>-5
s_outliercut = allStar["S_FE"]>-5
p_outliercut = allStar["P_FE"]>-5
ti_outliercut = allStar["Ti_FE"]>-5
cr_outliercut = allStar["Cr_FE"]>-5



combined_cut = feh_outliercut & o_outliercut &  c_outliercut & na_outliercut & mg_outliercut & si_outliercut & al_outliercut & p_outliercut & s_outliercut & ti_outliercut & cr_outliercut

we create a second order polynomial with the features

In [6]:
considered_parameters = ["Teff","logg","Fe_H","O_FE","C_FE","Na_FE","Mg_FE","Si_FE","S_FE","Al_FE","P_FE","Ni_FE"] 
y = vector.Vector(np.array([allStar[:n_data][combined_cut[:n_data]][param] for param in considered_parameters])[:,:n_data].T)
y_astronn = vector.AstroNNVector(allStar[:n_data][combined_cut[:n_data]],considered_parameters)

In [7]:
z = vector.Vector(z_ae.raw[combined_cut[:n_data]],order=2,interaction_only=False)

### Carrying out linear regression

We can study the linearity of our resultant representation by carrying linear and non-linear regression with the latents. We use the astroNN abundances as a gold-standard


In [ ]:
w = vector.LinearTransformation(z,y)
nonlinear_w = vector.NonLinearTransformation(z,y)
nonlinear_w.fit(n_epochs=1000)

err:4.650720596313477
err:2.7078516483306885
err:3.0880587100982666
err:2.5631775856018066
err:2.0298995971679688
err:2.8671586513519287
err:3.1524276733398438
err:2.718885898590088
err:2.3602218627929688
err:2.1818900108337402
err:1.320220708847046
err:1.8298054933547974
err:1.8254514932632446
err:2.4030823707580566
err:1.7834460735321045
err:1.6313027143478394
err:1.5513246059417725
err:1.4472222328186035
err:1.7882845401763916
err:1.4672973155975342
err:2.225541114807129
err:2.0458335876464844
err:1.320709228515625
err:1.3561758995056152
err:1.4391924142837524
err:1.6027601957321167
err:1.69294011592865
err:1.5790997743606567
err:1.3300119638442993
err:1.4132932424545288
err:1.0575675964355469
err:1.8921301364898682
err:1.4728171825408936
err:1.3563239574432373
err:1.2677292823791504
err:1.022010087966919
err:0.942077100276947
err:0.8567730784416199
err:0.9844799637794495
err:1.0343208312988281
err:1.579085111618042
err:1.1376851797103882
err:0.9792909622192383
err:1.709532260894775

err:0.03277747333049774
err:0.027753347530961037
err:0.03778060898184776
err:0.03114302270114422
err:0.03553473949432373
err:0.0319085419178009
err:0.02929121069610119
err:0.036875445395708084
err:0.036721695214509964
err:0.03203654661774635
err:0.030386704951524734
err:0.0317213274538517
err:0.032512977719306946
err:0.033034298568964005
err:0.048608385026454926
err:0.028305530548095703
err:0.035185568034648895
err:0.02588306926190853
err:0.03542720526456833
err:0.05038704350590706
err:0.04299856722354889
err:0.04586015269160271
err:0.046972185373306274
err:0.049826160073280334
err:0.03653116524219513
err:0.036753490567207336
err:0.05032508820295334
err:0.03477438539266586
err:0.02825566567480564
err:0.03777139261364937
err:0.0461563915014267
err:0.04652634635567665
err:0.035937268286943436
err:0.035100314766168594
err:0.04042418673634529
err:0.04200158268213272
err:0.027124693617224693
err:0.031313683837652206
err:0.03352024778723717
err:0.03412961587309837
err:0.03969039022922516
err

err:0.017949527129530907
err:0.019624000415205956
err:0.01906401664018631
err:0.023827997967600822
err:0.027000783011317253
err:0.024020396173000336
err:0.026492293924093246
err:0.018607791513204575
err:0.02091860957443714
err:0.02129681594669819
err:0.018303779885172844
err:0.023565277457237244
err:0.02004016935825348
err:0.024115856736898422
err:0.01972285658121109
err:0.020270168781280518
err:0.024409614503383636
err:0.025147587060928345
err:0.022676078602671623
err:0.020806020125746727
err:0.022088848054409027
err:0.022282179445028305
err:0.024182787165045738
err:0.032161612063646317
err:0.019562197849154472
err:0.023165680468082428
err:0.018233072012662888
err:0.027887830510735512
err:0.036623165011405945
err:0.03095545433461666
err:0.03375765681266785
err:0.03454450145363808
err:0.039332062005996704
err:0.026612048968672752
err:0.02684738114476204
err:0.03797311708331108
err:0.02453734166920185
err:0.020518898963928223
err:0.02925550378859043
err:0.034773316234350204
err:0.035199

err:0.019513044506311417
err:0.019052105024456978
err:0.022998735308647156
err:0.023096436634659767
err:0.02145274542272091
err:0.015593553893268108
err:0.014116735197603703
err:0.017419641837477684
err:0.02005007490515709
err:0.02118178829550743
err:0.014212030917406082
err:0.018421754240989685
err:0.017467482015490532
err:0.013064255937933922
err:0.015191437676548958
err:0.014764850959181786
err:0.018502837046980858
err:0.02121184766292572
err:0.017876198515295982
err:0.01972333714365959
err:0.014017251320183277
err:0.015456547960639
err:0.01562674529850483
err:0.013853131793439388
err:0.016754109412431717
err:0.015477827750146389
err:0.018869323655962944
err:0.015316472388803959
err:0.015942925587296486
err:0.018450552597641945
err:0.019425611943006516
err:0.017706960439682007
err:0.016254691407084465
err:0.017312686890363693
err:0.017134934663772583
err:0.0189246516674757
err:0.02609352581202984
err:0.015347536653280258
err:0.018343890085816383
err:0.014872169122099876
err:0.023302

err:0.02145724929869175
err:0.0157348420470953
err:0.013636006973683834
err:0.01626226305961609
err:0.01656368188560009
err:0.010903730057179928
err:0.015305444598197937
err:0.014453571289777756
err:0.01523533184081316
err:0.014489329420030117
err:0.016180969774723053
err:0.016235174611210823
err:0.01565062813460827
err:0.020253555849194527
err:0.0192368533462286
err:0.017356272786855698
err:0.012856091372668743
err:0.011831384152173996
err:0.014462725259363651
err:0.016743341460824013
err:0.017376260831952095
err:0.01122073270380497
err:0.015242383815348148
err:0.014751830138266087
err:0.01051595900207758
err:0.012639475055038929
err:0.012112612836062908
err:0.015160954557359219
err:0.01780221424996853
err:0.014810684137046337
err:0.016200734302401543
err:0.0114610455930233
err:0.0125522930175066
err:0.012439166195690632
err:0.011294824071228504
err:0.012796517461538315
err:0.012873126193881035
err:0.015864955261349678
err:0.01290887501090765
err:0.013306921347975731
err:0.01469015888

...

err:0.010846985504031181
err:0.010052199475467205
err:0.010562876239418983
err:0.009601525031030178
err:0.010931548662483692
err:0.011120210401713848
err:0.010778222233057022
err:0.01657005399465561
err:0.013548741117119789
err:0.011314800940454006
err:0.008602908812463284
err:0.00800450798124075
err:0.009621699340641499
err:0.011315794661641121
err:0.011378335766494274
err:0.007130405865609646
err:0.010522361844778061
err:0.01018260046839714
err:0.0070332749746739864
err:0.008781125769019127
err:0.007933440618216991
err:0.009772369638085365
err:0.012623388320207596
err:0.010511192493140697
err:0.011549952439963818
err:0.007425758056342602
err:0.008371138945221901
err:0.008498838171362877
err:0.007722060661762953
err:0.007652273401618004
err:0.008903576992452145
err:0.011131148785352707
err:0.009470014832913876
err:0.008985362015664577
err:0.009347742423415184
err:0.011285064741969109
err:0.00941964890807867
err:0.009326326660811901
err:0.009993442334234715
err:0.009839250706136227
err

err:0.01293670292943716
err:0.00942411832511425
err:0.009932407177984715
err:0.01000805851072073
err:0.011896689422428608
err:0.01393062248826027
err:0.010358915664255619
err:0.008748834952712059
err:0.010009200312197208
err:0.009623059071600437
err:0.0064925821498036385
err:0.010312945581972599
err:0.009529669769108295
err:0.009956763125956059
err:0.009000327438116074
err:0.010294917039573193
err:0.010419328697025776
err:0.010177805088460445
err:0.016016297042369843
err:0.01276537124067545
err:0.010509791783988476
err:0.007992817088961601
err:0.007456619758158922
err:0.008895698003470898
err:0.010623034089803696
err:0.01055922918021679
err:0.006660315673798323
err:0.009821888990700245
err:0.00957324355840683
err:0.006683748681098223
err:0.00821602251380682
err:0.007498325314372778
err:0.009085865691304207
err:0.011972230859100819
err:0.00986289232969284
err:0.01094990223646164
err:0.0069754114374518394
err:0.007833659648895264
err:0.008061475120484829
err:0.00733460346236825
err:0.007

err:0.018264668062329292
err:0.010771812871098518
err:0.011046255938708782
err:0.016357313841581345
err:0.011224834248423576
err:0.010229828767478466
err:0.012973830103874207
err:0.01344987377524376
err:0.013243063353002071
err:0.011729516088962555
err:0.010919907130300999
err:0.01378952618688345
err:0.012262220494449139
err:0.008936036378145218
err:0.009343305602669716
err:0.009434801526367664
err:0.011318610049784184
err:0.013065674342215061
err:0.009786187671124935
err:0.00827090535312891
err:0.009399664588272572
err:0.009025489911437035
err:0.00613828282803297
err:0.00981054175645113
err:0.009088326245546341
err:0.009452889673411846
err:0.008502054028213024
err:0.009775005280971527
err:0.009821348823606968
err:0.00961411651223898
err:0.015461403876543045
err:0.012101310305297375
err:0.009770120494067669
err:0.007492901291698217
err:0.006995563395321369
err:0.00830268394201994
err:0.01002148911356926
err:0.009864944033324718
err:0.006279288325458765
err:0.009232118725776672
err:0.00

err:0.008086573332548141
err:0.00856229942291975
err:0.008679557591676712
err:0.008725912310183048
err:0.01329993549734354
err:0.008160128258168697
err:0.009770012460649014
err:0.008449460379779339
err:0.011362683959305286
err:0.017120366916060448
err:0.01365335751324892
err:0.013837146572768688
err:0.013238485902547836
err:0.017274314537644386
err:0.010250451043248177
err:0.010482095181941986
err:0.015470952726900578
err:0.010585885494947433
err:0.009735926054418087
err:0.01223225798457861
err:0.012487377971410751
err:0.012488779611885548
err:0.01116322074085474
err:0.010307082906365395
err:0.013050599955022335
err:0.011697215959429741
err:0.008493890054523945
err:0.008808343671262264
err:0.008921333588659763
err:0.010799522511661053
err:0.012317231856286526
err:0.009300977922976017
err:0.007861414924263954
err:0.008888620883226395
err:0.008555280975997448
err:0.0058701192028820515
err:0.00935917068272829
err:0.008726255968213081
err:0.009035452269017696
err:0.008072554133832455
err:0

err:0.008738581091165543
err:0.0077834948897361755
err:0.00897120125591755
err:0.008935357443988323
err:0.008668551221489906
err:0.014594091102480888
err:0.01110792625695467
err:0.008653206750750542
err:0.006786259822547436
err:0.006313682068139315
err:0.007492301519960165
err:0.009159364737570286
err:0.00885460339486599
err:0.0057379635982215405
err:0.00840673316270113
err:0.008228259161114693
err:0.005999320652335882
err:0.00700202165171504
err:0.0066941422410309315
err:0.007847067900002003
err:0.010605541989207268
err:0.008438626304268837
err:0.009776926599442959
err:0.006115003488957882
err:0.0067708175629377365
err:0.007134954910725355
err:0.006542600225657225
err:0.006250564008951187
err:0.007264238316565752
err:0.008862020447850227
err:0.008034080266952515
err:0.007164137437939644
err:0.007484009489417076
err:0.009467635303735733
err:0.0076761022210121155
err:0.00777950044721365
err:0.008173284120857716
err:0.00835657212883234
err:0.008245332166552544
err:0.012768872082233429
er

In [ ]:
for i in range(len(considered_parameters)):
    plt.title(considered_parameters[i])
    nonlinear_std = np.std(nonlinear_w.predict(z).raw[:,i]-y.raw[:,i])
    linear_std = np.std(w.predict(z).raw[:,i]-y.raw[:,i])
    astronn_std = np.std(y_astronn.raw[:,i]-y.raw[:,i])


    plt.scatter(y.raw[:,i],w.predict(z).raw[:,i],s=0.2,label="linear:{:.4f}".format(linear_std))
    plt.scatter(y.raw[:,i],nonlinear_w.predict(z).raw[:,i],s=0.2,label="nonlinear:{:.4f}".format(nonlinear_std))
    plt.scatter(y.raw[:,i],y_astronn.raw[:,i],s=0.2,label="astronn:{:.4f}".format(astronn_std))

    plt.legend()
    plt.show()

In [ ]:
for i in range(len(considered_parameters)):
    plt.title(considered_parameters[i])
    nonlinear_std = np.std(nonlinear_w.predict(z).raw[:,i]-y.raw[:,i])
    linear_std = np.std(w.predict(z).raw[:,i]-y.raw[:,i])
    astronn_std = np.std(y_astronn.raw[:,i]-y.raw[:,i])


    plt.scatter(y.raw[:,i],w.predict(z).raw[:,i],s=0.2,label="linear:{:.4f}".format(linear_std))
    plt.scatter(y.raw[:,i],nonlinear_w.predict(z).raw[:,i],s=0.2,label="nonlinear:{:.4f}".format(nonlinear_std))
    plt.scatter(y.raw[:,i],y_astronn.raw[:,i],s=0.2,label="astronn:{:.4f}".format(astronn_std))

    plt.legend()
    plt.show()

## Occam

### Loading Occam data

In [ ]:
occam = load("occam")
allStar_occam = occam["allStar"]
dataset_occam = AspcapDataset(filename="aspcap_occam",recenter=True,tensor_type=torch.FloatTensor,filling_dataset=dataset.dataset["aspcap"])

In [ ]:
occam_cluster_idxs = occam["cluster_idxs"]
z_occam = vector.OccamLatentVector(occam_cluster_idxs,dataset=dataset_occam,autoencoder = autoencoder,n_data = len(dataset_occam))


In what follows, we show a simple loop for identifying, using the reconstruction loss, any outlier spectra which are not well captured by the autoencoder

In [ ]:
bad_indexes = []
for i in range(len(z_occam.raw)):
    err = np.sqrt(np.sum(((z_occam.get_x(i)[mask_elem.astype(bool)]-z_occam.get_x_pred(i))**2).detach().numpy()))
    print(err)
    if err>0.5:
        print(err)
        print(i)
        bad_indexes.append(i)
        plt.plot(z_occam.get_x(i)[mask_elem.astype(bool)])
        plt.plot(z_occam.get_x_pred(i))
        #z_occam.plot(i)
        plt.show()

We leave a commentedout piece of code for deleting any outlier spectra, if judged necessary.

retained= np.delete(np.arange(len(z_occam.raw)),bad_indexes)
retained

In [ ]:
z_occam = vector.OccamLatentVector(occam_cluster_idxs,raw=z_occam.raw,order=2,interaction_only=False)

### PCA investigation

The PCA procedure involves two steps:

1. Transform the latent representations to a whitened representation. In the whitened representation, all factors of variation in the full dataset have appropriately equal variance. 

2. Find those factors of variations, in the whitened representation, which have lowest intracluster variance. 

In [ ]:
whitener = PCA(n_components=z.raw.shape[1],whiten=True)#z.raw.shape[1],whiten=True)


In [ ]:
whitener.fit(z.centered)

now that we have learned a whitener. We transform our dataset to the new whitened space

In [ ]:
w_z_c = whitener.transform(z.centered)
w_z_occam_c = whitener.transform(z_occam.cluster_centered)

Normally here the variance in all the directions, for the full dataset, should be roughly equal. We can check this

In [ ]:
pca = PCA(n_components=z.raw.shape[1])
pca.fit(w_z_c[:2500,:])
print(pca.explained_variance_)

Hopefully when we look at clusters, there should be some directions along which there are not really any variations

In [ ]:
pca.fit(w_z_occam_c)
print(pca.explained_variance_)

### Assessing Open clusters

Our latent representations merit is evaluated by looking at how much variance is found within clusters compared to within full dataset. Because we don't want our representation to be influenced by outlier datapoints (where the model has failed), we threshold the most outlier datapoints.

**In this exercise, we compare Fe/H, the most well estimated parameter, to our latents smallest direction of variation**

In [ ]:
dim = 18
zc_b = vector.project(w_z_c,pca.components_[dim][None,:])
zoccam_b = vector.project(w_z_occam_c,pca.components_[dim][None,:])

In [ ]:
summarize_representation(zc_b,zoccam_b,0.2,0.2)

### Comparing to raw abundances

We can compare our representations expressiveness to the raw abundances.

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [ ]:
abundances_occam = []
elem = "Fe_H"
elem_idx = considered_parameters.index(elem)

for cluster in z_occam.registry:
    clust_idxs = z_occam.registry[cluster]
    abundances_occam.append(allStar_occam[clust_idxs][elem]-np.mean(allStar_occam[clust_idxs][elem]))
    #print(z_occam.dataset.allStar[clust_idxs][elem]-z_occam.dataset.allStar[clust_idxs][elem])
abundances_occam = np.array(flatten(abundances_occam))

In [ ]:
abundances_all = y.raw[:,elem_idx]-np.mean(y.raw[:,elem_idx])

In [ ]:
summarize_representation(abundances_all,abundances_occam,0.2,0.2)

### Comparing to astroNN

In [ ]:
y_astronn = vector.AstroNNVector(allStar[:n_data][combined_cut[:n_data]],considered_parameters)
y_astronn_occam = vector.AstroNNVector(allStar_occam,considered_parameters)

In [ ]:
abundances_nn_occam = []
elem = "Fe_H"
elem_idx = considered_parameters.index(elem)

for cluster in z_occam.registry:
    clust_idxs = z_occam.registry[cluster]
    abundances_nn_occam.append(y_astronn_occam.raw[clust_idxs,elem_idx]-np.mean(y_astronn_occam.raw[clust_idxs,elem_idx]))
    #print(z_occam.dataset.allStar[clust_idxs][elem]-z_occam.dataset.allStar[clust_idxs][elem])
abundances_nn_occam = np.array(flatten(abundances_nn_occam))

In [ ]:
abundances_nn_all  =y_astronn.raw[:,elem_idx]-np.mean(y_astronn.raw[:,elem_idx])


In [ ]:
summarize_representation(abundances_nn_all,abundances_nn_occam,0.2,0.2)